In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

In [ ]:

OpenTargets = pd.read_csv("/Users/talgalper/Desktop/OpenTargets_unique_drug.csv")
len(OpenTargets)


In [ ]:
# extract NCT IDs from URL
def extract_nct_id(url):
    parts = url.split('/')
    for part in parts:
        if part.startswith('NCT'):
            return part
    return None

NCT_IDs = OpenTargets['URL'].apply(extract_nct_id)
# remove NA/missing values
NCT_IDs = NCT_IDs.dropna()
len(NCT_IDs)

In [ ]:
import pandas as pd
nct_ids = []
brief_summaries = []

# Loop over each NCT ID and fetch the brief summary
for nct_id in tqdm(NCT_IDs):
    url = f"https://clinicaltrials.gov/api/v2/studies/{nct_id}?format=json&fields=BriefSummary"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if 'protocolSection' in data and 'descriptionModule' in data['protocolSection'] and 'briefSummary' in data['protocolSection']['descriptionModule']:
            brief_summary = data['protocolSection']['descriptionModule']['briefSummary']
        else:
            brief_summary = "No summary available"
    else:
        brief_summary = f"Error: {response.status_code}"
    
    nct_ids.append(nct_id)
    brief_summaries.append(brief_summary)

In [ ]:
summary_df = pd.DataFrame({'NCT ID': nct_ids, 'Brief Summary': brief_summaries})

In [ ]:
summary_df.to_csv("/Users/talgalper/Desktop/NCT_summaries.csv", index=False)